### 1. 상자로서의 변수

#### 1) Variable 클래스 구현

In [3]:
import numpy as np

In [4]:
class Variable:
    def __init__(self, data):
        self.data = data

In [5]:
data = np.array(1.0)

In [6]:
x = Variable(data)

In [7]:
print(x.data) # x는 데이터 자체가 아니라 데이터를 담은 상자

1.0


#### cf) 넘파이의 다차원 배열
- .ndim : 차원 수

In [8]:
x = np.array(1)
x.ndim

0

In [9]:
x = np.array([1,2,3])
x.ndim

1

In [10]:
x = np.array([[1,2,3],
              [4,5,5]])
x.ndim

2

### 2. 변수를 낳는 함수

#### 1) Function 클래스 구현
- Variable 인스턴스(클래스 객체의 인스턴스)를 입력받아 Variable 인스턴스를 출력


- \_\_call\_\_ 매서드 : 특수 매서드로 f = Function() 형태로 함수의 인스턴스를 변수 f에 대입해두고, 나중에 f(...) 형태로 \_\_call\_\_ 매서드를 호출할 수 있음

In [11]:
class Function:
    def __call__(self, input):
        x = input.data # 데이터를 꺼낸다
        y = x ** 2 # 실제 계산
        output = Variable(y) # Variable 형태로 되돌린다.
        return output

In [12]:
x = Variable(np.array(1.0))
f = Function()
y = f(x)

In [13]:
print(type(y)) # type() 함수는 객체의 클래스를 알려줌 (y : Variable)
print(y.data)

<class '__main__.Variable'>
1.0


#### 2) Function 클래스 응용하기 (함수를 클래스 객체로 <- 추가 가능하게)
- \_\_call\_\_ 매서드에서는 '데이터 찾기'와 '계산 결과를 Variable에 포장하기'만 수행
- 구체적인 계산은 forward 매서드에서

In [14]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) # 구체적인 계산은 forward 매서드에서 한다
        output = Variable(y)
        return output
    
    def forward(self, x):
        raise NotImplementedError() # 아직 구현되지 않은 부분입니다

In [15]:
# Function 클래스에 상속하여 입력값을 제곱하는 클래스 구현

class Square(Function):
    def forward(self, x):
        return x ** 2

In [16]:
x = Variable(np.array(1.0))
f = Square()
y = f(x)

In [17]:
print(type(y))
print(y.data)

<class '__main__.Variable'>
1.0


### 3. 함수 연결

#### 1) Exp 함수 구현

In [18]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

#### 2) 함수 연결(합성 함수)

In [19]:
A = Square()
B = Exp()
C = Square()

In [20]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
y.data

1.648721270700128

### 4. 수치 미분
- h = 0.0001로 두고 함수의 변화율을 계산
- 오차를 줄이기 위해 중앙차분 이용 : f(x+h) - f(x-h) / 2h

In [21]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return(y1.data - y0.data) / (2 * eps)

In [23]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
dy

4.000000000004

#### 합성 함수의 미분
- y = (e^(x^2))^2 

In [24]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return(C(B(A(x))))

In [25]:
x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
dy

3.2974426293330694

### 5. 역전파 이론
![ ](img/5_2.png)
![ ](img/5_4.png)

### 6. 수동 역전파

#### 1) Variable 클래스 추가 구현

In [26]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None

#### 2) Function 클래스 추가 구현

In [64]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x) 
        output = Variable(y)
        self.input = input   # 입력 변수를 기억한다 -- 추가된 부분
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy):  # -- 추가된 부분
        raise NotImplementedError()

In [65]:
class Square(Function):
    def forward(self, x):
        return x ** 2
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [66]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

#### 3) 역전파 구현

In [67]:
A = Square()
B = Exp()
C = Square()

In [68]:
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [69]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

In [70]:
C.input.data

1.2840254166877414

### 7. 역전파 자동화

#### 1) 자동화 시작
- 함수의 입장에서의 변수는 입력과 출력
- 변수 관점에서의 함수는 창조자

    => 이 관계를 맺어주기

In [47]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None        # 여기서부터는 추가된 부분
        
    def set_creator(self, func):
        self.creator = func

In [48]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        
        output.set_creator(self) # 출력 변수에 창조자를 설정한다
        self.input = input
        self.output = output     # 출력도 저장한다
        return output

In [51]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [52]:
# 계산 그래프의 노드들을 거꾸로 거슬로 올라간다
assert y.creator == C  # assert문은 True가 아니면 예외 발생
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

#### 2) 역전파 도전

In [53]:
y.grad = np.array(1.0)

C = y.creator # 1. 함수를 가져온다
b = C.input   # 2. 함수의 입력을 가져온다
b.grad = C.backward(y.grad) # 3. 함수의 backward 메서드를 호출한다

In [54]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [58]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
x.grad

3.297442541400256

#### 3) backward 메서드 추가

In [60]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None        # 여기서부터는 추가된 부분
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator  # 1. 함수를 가져온다
        
        if f is not None:
            x = f.input  # 2. 함수의 입력을 가져온다
            x.grad = f.backward(self.grad) # 3. 함수의 backward 메서드를 호출한다
            x.backward() # 하나 앞 변수의 backward 메서드를 호출한다

In [63]:
# 역전파

y.grad = np.array(1.0)
y.backward()
x.grad

3.297442541400256